In [2]:
import re
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install factor_analyzer

#### Loading in data 
after initial load, no need to run the immediate cell and instead, load acc_df from the pkl file.

In [19]:
# need to obtain the tar file from lending club loan data on Kaggle
filepath = './'
folders = os.listdir(filepath)
acc_df = pd.read_csv("./accepted_2007_to_2018Q4.csv.gz")
acc_df.to_pickle('acc_df_cached.pkl')

/var/folders/tz/0pg3_6s57f39kds6_g4kpl680000gn/T/ipykernel_62289/224753027.py:3: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  acc_df = pd.read_csv("./accepted_2007_to_2018Q4.csv.gz")


In [59]:
acc_df = pd.read_pickle('acc_df_cached.pkl') # read from current directory, this is faster

In [60]:
acc_df.columns[0:50]

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d'],
      dtype='object')

In [61]:
acc_df.columns[100:]

Index(['num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies',
       'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_flag', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_

In [62]:
acc_df.columns[50:100]

Index(['last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_

In [63]:
acc_df.shape
acc_df.head().T
acc_df["loan_status"].value_counts()

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

### Data Cleaning
Three categories of loan status results
* Fully Paid
* Late (grace period, 16-30, 31-120)
* Charged off / default

In [69]:
# different combinations of explanatory columns
# * watch out for high collinearity
explore_columns = {
    1: ["tax_liens", "collections_12_mths_ex_med", "pub_rec_bankruptcies", "num_accts_ever_120_pd"],
    2: ["tax_liens", "collections_12_mths_ex_med", "pub_rec_bankruptcies", "pct_tl_nvr_dlq", "num_accts_ever_120_pd"],
    3: ["tax_liens", "pub_rec_bankruptcies"],
    4: [],
}

# list of relevant columns
info = ["dti_joint", "verification_status_joint", "grade", "sub_grade"]
dependent = ["loan_status"]
explanatory = explore_columns[1]

In [101]:
# filter for relevant columns
filtered_df = acc_df[info + dependent + explanatory]

# data cleaning - given that NAs make up a small portion of the overall data, we drop NAs
print(f"count before drop na: {len(filtered_df)}, count after drop na: {len(filtered_df.dropna(subset=explanatory+dependent) )}")
filtered_df = filtered_df.dropna(subset=explanatory+dependent) # after dropping, 2260701 -> 2106856

# split based on loan status
late_df = filtered_df[filtered_df['loan_status'].isin(["Late (16-30 days)", "Late (31-120 days)", "In Grace Period"])]
defaulted_df = filtered_df[filtered_df['loan_status'].isin(["Default", "Charged Off"])]
paid_df = filtered_df[filtered_df['loan_status'] == "Fully Paid"]

# map loan status to an indicator variable TODO - decide how to categorize lates
value_map = {'Fully Paid': 0, 'Default': 1, 'Charged Off': 1, "Late (16-30 days)": 1, "Late (31-120 days)": 1, "In Grace Period": 0}
filtered_df["default_ind"] = filtered_df['loan_status'].map(value_map)

count before drop na: 2260701, count after drop na: 2190392


In [102]:
filtered_df.head()

,dti_joint,verification_status_joint,grade,sub_grade,loan_status,tax_liens,collections_12_mths_ex_med,pub_rec_bankruptcies,num_accts_ever_120_pd,default_ind
0,NaN,NaN,C,C4,Fully Paid,0.0,0.0,0.0,2.0,0.0
1,NaN,NaN,C,C1,Fully Paid,0.0,0.0,0.0,0.0,0.0
2,13.85,Not Verified,B,B4,Fully Paid,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,C,C5,Current,0.0,0.0,0.0,0.0,NaN
4,NaN,NaN,F,F1,Fully Paid,0.0,0.0,0.0,0.0,0.0


### Latent Variables

columns of interest (explantory variates):

a factor model resembles a series of regressions to predict the observed variables from the unobserved.

Each of our response variables X is predicted as a linear function of the unobserved common factors

we want one unobserved common factor, responsibility (fr)

mean 0 and variance 1

In [103]:
# covariance matrix
filtered_df[explanatory].cov()

,tax_liens,collections_12_mths_ex_med,pub_rec_bankruptcies,num_accts_ever_120_pd
tax_liens,0.147025,0.000604,0.004553,0.006207
collections_12_mths_ex_med,0.000604,0.023462,-0.000265,0.009988
pub_rec_bankruptcies,0.004553,-0.000265,0.135719,-0.015937
num_accts_ever_120_pd,0.006207,0.009988,-0.015937,1.823379


In [104]:
# # for linear regression - use this code if we are predicting continuous response variates (?)
# from sklearn import linear_model
# regr = linear_model.LinearRegression()

# import statsmodels.api as sm

# subset_expl = ["pub_rec_bankruptcies","tax_liens"]
# X = filtered_df[subset_expl]
# y = filtered_df[["default_likelihood"]]
# X = sm.add_constant(X) # add intercept term

# # Define and fit the model
# model = sm.OLS(y, X)
# results = model.fit()
# print(results.summary())

### One Factor Confirmatory Factor Analysis
determine if our selected columns of interests do share an underlying factor

https://www.datacamp.com/tutorial/introduction-factor-analysis

In [105]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

In [106]:
subset_explanatory = ["tax_liens", "collections_12_mths_ex_med", "pub_rec_bankruptcies", "num_accts_ever_120_pd"]
df = filtered_df[subset_explanatory]

In [107]:
# perform an adequacy test
chi_square_value,p_value=calculate_bartlett_sphericity(df)
chi_square_value, p_value

# observe that there is a very small p-value, thus this test passes.

(10245.436971871439, 0.0)

In [108]:
kmo_all,kmo_model=calculate_kmo(df)
kmo_model
# value of 50 is not ideal, but maybe we can go with it

0.49940419105860423

In [109]:
fa = FactorAnalyzer(n_factors=1,rotation='varimax')
fa.fit(df)
factor_df=pd.DataFrame(fa.loadings_,index=df.columns)
print(factor_df)

                                   0
tax_liens                  -0.011449
collections_12_mths_ex_med -0.048623
pub_rec_bankruptcies        0.031897
num_accts_ever_120_pd      -0.997498


/Users/vivian/anaconda3/lib/python3.11/site-packages/factor_analyzer/factor_analyzer.py:663: UserWarning: No rotation will be performed when the number of factors equals 1.
  warnings.warn(


In [110]:
fa.get_factor_variance()

(array([0.99851586]), array([0.24962897]), array([0.24962897]))

In [111]:
print(f"total explained variance by factor: {sum(sum(fa.get_factor_variance()))}")

total explained variance by factor: 1.497773790023899


In [112]:
fa.loadings_

array([[-0.01144864],
       [-0.04862257],
       [ 0.03189682],
       [-0.99749848]])

In [113]:
# get factor scores (documentation: https://readthedocs.org/projects/factor-analyzer/downloads/pdf/latest/)
fa.transform(df.values)

array([[-1.10787631],
       [ 0.36952309],
       [ 0.36952309],
       ...,
       [-4.06267512],
       [-0.36939128],
       [-2.58527571]])

### Logistic Regression

In [114]:
# create a responsibility measure based on given columns 
def measure_responsibility(df, columns):
    df["responsibility"] = 0

    # standardize each column in columns and sum to responsibility
    for col in columns:
        # df[col].fillna(df[col].median()) # median imputation
        df[f'{col}_standardized'] = (df[col] - df[col].mean()) / df[col].std()
        df["responsibility"] = df["responsibility"] + df[f'{col}_standardized']  # Add each standardized column
    return df["responsibility"]

In [115]:
filtered_df["responsibility"] = measure_responsibility(filtered_df, explanatory)

In [116]:
import numpy as np
import statsmodels.api as sm

In [117]:
print(f"count before drop na: {len(filtered_df)}, ")
print(f"count after drop na: {len(filtered_df.dropna(subset=['default_ind']))}") # this na drop is a bit high

filtered_df = filtered_df.dropna(subset=["default_ind"]) 
x = np.array(filtered_df["responsibility"])
y = np.array(filtered_df["default_ind"])          # response
x = sm.add_constant(x)  

model = sm.Logit(y, x).fit()

count before drop na: 2190392, 
count after drop na: 1312075
Optimization terminated successfully.
         Current function value: 0.522053
         Iterations 5


In [118]:
# the error residual term is significant? - much is unexplained
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:              1312075
Model:                          Logit   Df Residuals:                  1312073
Method:                           MLE   Df Model:                            1
Date:                Sat, 26 Oct 2024   Pseudo R-squ.:               0.0008062
Time:                        11:08:29   Log-Likelihood:            -6.8497e+05
converged:                       True   LL-Null:                   -6.8553e+05
Covariance Type:            nonrobust   LLR p-value:                2.279e-242
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2887      0.002   -606.850      0.000      -1.293      -1.285
x1             0.0325      0.001     33.404      0.000       0.031       0.034
==============================================================================
"""